# Add Registrar's Notation to all existing companies

<b> Purpose: Add Registrar's Notation filing to all existing BENs.</b>

This is a one time (python) script to be run at a given date/time.<br>
Set the configuration (client_id, client_secret, url(s)) for a scpecific environment.<br>
Get access token for authorization.


In [ ]:
import requests
import os
from datetime import datetime

# token_url, client_id, client_secret, base_url - update based on environment
token_url = os.getenv('ACCOUNT_SVC_AUTH_URL')
client_id = os.getenv('ACCOUNT_SVC_CLIENT_ID')
client_secret = os.getenv('ACCOUNT_SVC_CLIENT_SECRET')
base_url = os.getenv('LEGAL_API_BASE_URL')

header = {
    "Content-Type": "application/x-www-form-urlencoded"
}

data = 'grant_type=client_credentials'

res = requests.post(token_url, data, auth=(client_id, client_secret), headers=header)

# Check the status code of the response
if res.status_code == 200:
    print(f"Access token returned successfully : {base_url}")
    token = res.json()["access_token"]
else:
    print(f"Failed to make POST request. Status code: {res.status_code}")
    print(res.text)  # Print the error message if the request fails



Call API (POST) endpoint to create Registrar's Notation filing for businesses.

In [ ]:
from urllib.parse import urljoin
from rn_output import rn_businesses

current_date = datetime.now().date().isoformat()
headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Bearer ' + token
}

failed_identifiers = []

# loop through list of businesses to create filing
for business in rn_businesses:
    filing_data = {
        "filing": {
            "header": {
                "name": "registrarsNotation",
                "date": current_date,
                "certifiedBy": "system"
            },
            "business": {
                "identifier": business,
                "legalType": "BEN"
            },
            "registrarsNotation": {
                "orderDetails": "BC benefit company statement contained in notice of articles as required under " + 
                "section 51.992 of the Business Corporations Act corrected from " +
                "\"This company is a benefit company and, as such, has purposes that include conducting its business " +
                " in a responsible and sustainable manner and promoting one or more public benefits\" to " + 
                "\"This company is a benefit company and, as such, is committed to conducting its business in a " + 
                "responsible and sustainable manner and promoting one or more public benefits\"."
            }
        }
    }

    filing_url = urljoin(base_url, f"/api/v2/businesses/{business}/filings")
    response = requests.post(filing_url, headers=headers, json=filing_data)

    # Check the status code of the response
    if response.status_code == 201:
        print(f"Registrars Notation created successfully for {business}")
    else:
        failed_identifiers.append(business)
        print(f"Failed to make POST request. Status code: {response.status_code}")
        print(response.text)  # Print the error message if the request fails
failed_identifiers = []
